# Goal: See how much of an effect our wasting treatment and prevention are having

Model version: `v4.1_wasting_treatment`

It looks like we're averting around 1% of total DALYs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Mon Sep 27 12:20:18 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
csr.get_count_data_path(4.1)

'/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.1_wasting_treatment/ciff_sam/2021_09_24_16_36_30/count_data/'

# Load data and compute total person time

Note that this model run was only partially complete.

In [4]:
data = csr.VivariumResults.cleaned_from_model_spec(4.1)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count']

In [5]:
data.compute_total_person_time()
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count',
 'person_time']

# Look at total DALYs in each scenario and calculate averted DALYs

Start with YLLs

In [6]:
data.ylls

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,2577.145536,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,533.187801,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,177.735076,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
51836,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,susceptible_to_child_wasting,2_to_4
51837,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,mild_child_wasting,2_to_4
51838,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,moderate_acute_malnutrition,2_to_4
51839,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,687.248870,severe_acute_malnutrition,2_to_4


In [7]:
averted_ylls = vop.averted(data.ylls, 'baseline')
averted_ylls

,sex,year,cause,measure,input_draw,wasting_state,age,scenario,subtracted_from,value
0,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,1-5_months,sqlns,baseline,9.094947e-13
1,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,1-5_months,wasting_treatment,baseline,9.094947e-13
2,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,12_to_23_months,sqlns,baseline,-4.547474e-13
3,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,12_to_23_months,wasting_treatment,baseline,-4.547474e-13
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,early_neonatal,sqlns,baseline,0.000000e+00
34557,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,early_neonatal,wasting_treatment,baseline,0.000000e+00
34558,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,late_neonatal,sqlns,baseline,0.000000e+00
34559,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,late_neonatal,wasting_treatment,baseline,0.000000e+00


# Look at averted YLLs and total YLLs in each scenario

Averted YLLs more than double when implementing prevention (SQLNS) in addition to wasting treatment.

We are only averting around 1% of total DALYs in each scenario.

In [9]:
# Stratifying by nothing tallies results over all strata for each scenario and draw
# Calling vop.describe() summarizes the distribution over draws
vop.describe(vop.stratify(averted_ylls, []))

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
sqlns,12.0,61270.188012,16260.721246,33186.641288,33640.539217,62351.891794,83319.043413,84433.926547
wasting_treatment,12.0,29135.866613,6712.971732,18020.910774,18477.352351,30593.500321,37323.844437,37872.478553


In [10]:
vop.describe(vop.stratify(data.ylls, []))

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,4.832559e+06,633220.091632,3.437005e+06,3.587341e+06,5.044998e+06,5.567974e+06,5.639034e+06
sqlns,12.0,4.771289e+06,621658.617388,3.403818e+06,3.553700e+06,4.984684e+06,5.495644e+06,5.569758e+06
wasting_treatment,12.0,4.803423e+06,629510.526416,3.418984e+06,3.568864e+06,5.017030e+06,5.535069e+06,5.605000e+06


In [30]:
# Fraction of total DALYs averted in wasting_treatment and sqlns scenarios
# Except I'm dividing by YLLs in each scenario instead of YLLs in baseline...
vop.ratio(
    averted_ylls,
    data.ylls,
    strata=[],
    multiplier=100, # convert to percent DALYs averted
).groupby('scenario').mean()

,input_draw,value,multiplier
scenario,,,
baseline,507.916667,NaN,100.0
sqlns,507.916667,1.272811,100.0
wasting_treatment,507.916667,0.606200,100.0


# Stratify overall YLLs and YLDs by wasting state

Most DALYs are in the sucecptible and mild states, whereas our treatments affect the moderate and severe states.

In [12]:
vop.describe(vop.stratify(data.ylls, ['wasting_state']))['mean'].unstack()

wasting_state,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
scenario,,,,
baseline,1.005201e+06,594687.868478,255739.337575,2.976931e+06
sqlns,1.030257e+06,532236.707433,230281.820205,2.978513e+06
wasting_treatment,1.022599e+06,559300.317286,239327.195029,2.982197e+06


In [13]:
vop.describe(vop.stratify(data.ylds, ['wasting_state']))['mean'].unstack()

wasting_state,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
scenario,,,,
baseline,4507.579261,3619.841143,15008.435059,8481.051990
sqlns,4671.998035,2901.379839,12314.025598,8399.411969
wasting_treatment,4675.930915,3202.261920,13231.158730,8522.035674


## What's the right way to compute averted YLLs by wasting state? Probably none of these...

### Computation from Model 4.0 notebook: Total YLLs averted over YLLs in MAM and SAM

As a fraction of MAM and SAM DALYS, the averted DALYs are more like 3% or 7% (but note that the numerators are not stratified by wasting state).

In [48]:
# fraction of MAM and SAM DALYs averted in wasting_treatment and sqlns scenarios
# Total YLLs averted over (baseline YLLs in MAM + baseline YLLs in SAM)
np.array([20735.562130, 43164.853815]) / (416103.353244+179741.929651)

array([0.03480025, 0.07244306])

### Compute total averted YLLs over YLLs stratified by wasting state

This probably doesn't make sense...

In [35]:
# Total YLLs averted over YLLs in each wasting state
vop.ratio(
    averted_ylls,
    data.ylls,
    strata=[],
    denominator_broadcast='wasting_state',
    multiplier=100, # convert to percent DALYs averted
).groupby(['scenario', 'wasting_state']).mean().dropna()

input_draw      value  \
scenario          wasting_state                                         
sqlns             mild_child_wasting            507.916667   5.895098   
                  moderate_acute_malnutrition   507.916667  11.386588   
                  severe_acute_malnutrition     507.916667  26.281775   
                  susceptible_to_child_wasting  507.916667   2.042025   
wasting_treatment mild_child_wasting            507.916667   2.847968   
                  moderate_acute_malnutrition   507.916667   5.209059   
                  severe_acute_malnutrition     507.916667  12.192543   
                  susceptible_to_child_wasting  507.916667   0.977055   

                                                multiplier  
scenario          wasting_state                             
sqlns             mild_child_wasting                 100.0  
                  moderate_acute_malnutrition        100.0  
                  severe_acute_malnutrition          100.0  
                  susceptible_to_child_wasting       100.0  
wasting_treatment mild_child_wasting                 100.0  
                  moderate_acute_malnutrition        100.0  
                  severe_acute_malnutrition          100.0  
                  susceptible_to_child_wasting       100.0

In [37]:
# YLLs averted in each wasting state over total YLLs in that state
vop.ratio(
    averted_ylls,
    data.ylls,
    strata=['wasting_state'],
    multiplier=100, # convert to percent DALYs averted
).groupby(['scenario', 'wasting_state']).mean().dropna()

input_draw      value  \
scenario          wasting_state                                         
sqlns             mild_child_wasting            507.916667  -2.428693   
                  moderate_acute_malnutrition   507.916667  11.663491   
                  severe_acute_malnutrition     507.916667  11.017079   
                  susceptible_to_child_wasting  507.916667  -0.054151   
wasting_treatment mild_child_wasting            507.916667  -1.707304   
                  moderate_acute_malnutrition   507.916667   6.356986   
                  severe_acute_malnutrition     507.916667   6.888634   
                  susceptible_to_child_wasting  507.916667  -0.177275   

                                                multiplier  
scenario          wasting_state                             
sqlns             mild_child_wasting                 100.0  
                  moderate_acute_malnutrition        100.0  
                  severe_acute_malnutrition          100.0  
                  susceptible_to_child_wasting       100.0  
wasting_treatment mild_child_wasting                 100.0  
                  moderate_acute_malnutrition        100.0  
                  severe_acute_malnutrition          100.0  
                  susceptible_to_child_wasting       100.0

# Compute fraction of total population in each wasting state in each scenario

We want to see how much our interventions decrease the prevalence of wasting.

In [14]:
data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,201.939767,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,207.442847,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,covered,2_to_4
34557,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,uncovered,2_to_4
34558,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,198117.045859,uncovered,covered,2_to_4
34559,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,21226.153320,uncovered,uncovered,2_to_4


## Get person time in each wasting state and total person time, filter out pre-intervention years

In [15]:
pt_by_wasting_state = vop.stratify(
    data.wasting_state_person_time.query("year > '2022'"), ['wasting_state', 'measure'])
pt_by_wasting_state

,wasting_state,measure,input_draw,scenario,value
0,mild_child_wasting,state_person_time,29,baseline,8.971736e+05
1,mild_child_wasting,state_person_time,29,sqlns,9.688046e+05
2,mild_child_wasting,state_person_time,29,wasting_treatment,9.373038e+05
3,mild_child_wasting,state_person_time,223,baseline,8.954353e+05
...,...,...,...,...,...
140,susceptible_to_child_wasting,state_person_time,829,wasting_treatment,2.903149e+06
141,susceptible_to_child_wasting,state_person_time,946,baseline,2.860194e+06
142,susceptible_to_child_wasting,state_person_time,946,sqlns,2.894737e+06
143,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,2.882659e+06


In [16]:
data.person_time

,year,sex,age,input_draw,scenario,value,measure
0,2022,female,1-5_months,29,baseline,43985.986311,person_time
1,2022,female,1-5_months,29,sqlns,43985.986311,person_time
2,2022,female,1-5_months,29,wasting_treatment,43985.986311,person_time
3,2022,female,1-5_months,223,baseline,43770.932238,person_time
...,...,...,...,...,...,...,...
2516,2023,male,all_ages,946,wasting_treatment,530905.464750,person_time
2517,2024,male,all_ages,946,wasting_treatment,536050.442163,person_time
2518,2025,male,all_ages,946,wasting_treatment,536439.126626,person_time
2519,2026,male,all_ages,946,wasting_treatment,534609.148528,person_time


In [17]:
pt = vop.stratify(data.person_time.query("age != 'all_ages' and year > '2022'"), ['measure'])
pt

,measure,input_draw,scenario,value
0,person_time,29,baseline,4.178410e+06
1,person_time,29,sqlns,4.179728e+06
2,person_time,29,wasting_treatment,4.179065e+06
3,person_time,223,baseline,4.162766e+06
...,...,...,...,...
32,person_time,829,wasting_treatment,4.208040e+06
33,person_time,946,baseline,4.173020e+06
34,person_time,946,sqlns,4.173859e+06
35,person_time,946,wasting_treatment,4.173466e+06


## Calculate prevalence of each wasting state

In [18]:
pop_fraction_by_wasting_state = vop.ratio(
    pt_by_wasting_state,
    pt,
    strata = [],
    numerator_broadcast='wasting_state'
)
pop_fraction_by_wasting_state

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,0.214716,state_person_time,person_time,1
1,29,baseline,moderate_acute_malnutrition,0.080046,state_person_time,person_time,1
2,29,baseline,severe_acute_malnutrition,0.019478,state_person_time,person_time,1
3,29,baseline,susceptible_to_child_wasting,0.685760,state_person_time,person_time,1
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.223208,state_person_time,person_time,1
141,946,wasting_treatment,moderate_acute_malnutrition,0.069967,state_person_time,person_time,1
142,946,wasting_treatment,severe_acute_malnutrition,0.016114,state_person_time,person_time,1
143,946,wasting_treatment,susceptible_to_child_wasting,0.690711,state_person_time,person_time,1


## It looks like SAM and MAM prevalence are going _up_ in our alternative scenarios!

This is likely due to a bug in the observers. Rajan says in a [Slack message from Wed 9/22, 2021](https://ihme.slack.com/archives/C018BLX2JKT/p1632347346165200):

> **Rajan Mudambi**  2:49 PM
>
> @Nathaniel Blair-Stahn Just as a heads up, in the partial runs we got yesterday the alternative wasting scenarios doesn't capture any wasting person time or transitions in the covered population due to a bug I found today. The baseline scenario should be fine as well as the other count data. I have a new run underway with the fix.

In [19]:
vop.describe(pop_fraction_by_wasting_state)['mean'].unstack()

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         1          state_person_time baseline                     0.215168   
                                                 sqlns                        0.230216   
                                                 wasting_treatment            0.224093   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                              0.079556   
                                                 sqlns                                 0.061496   
                                                 wasting_treatment                     0.069078   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         1          state_person_time baseline                            0.018876   
                                                 sqlns                               0.014601   
                                                 wasting_treatment                   0.016015   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                               0.686399  
                                                 sqlns                                  0.693688  
                                                 wasting_treatment                      0.690815

In [38]:
vop.describe(pop_fraction_by_wasting_state)['mean'].unstack().sum(axis=1)

denominator_measure  multiplier  numerator_measure  scenario         
person_time          1           state_person_time  baseline             1.0
                                                    sqlns                1.0
                                                    wasting_treatment    1.0
dtype: float64

# Recompute wasting prevalence to verify

In [45]:
query_string = "year>'2022' and age!='all_ages'"
wasting_prevalence = vop.ratio(
    data.wasting_state_person_time.query(query_string),
    data.person_time.query(query_string),
    strata=[],
    numerator_broadcast='wasting_state',
    multiplier=100, # Convert units to percent
)
wasting_prevalence

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,21.471650,state_person_time,person_time,100
1,29,baseline,moderate_acute_malnutrition,8.004558,state_person_time,person_time,100
2,29,baseline,severe_acute_malnutrition,1.947800,state_person_time,person_time,100
3,29,baseline,susceptible_to_child_wasting,68.575993,state_person_time,person_time,100
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,22.320811,state_person_time,person_time,100
141,946,wasting_treatment,moderate_acute_malnutrition,6.996664,state_person_time,person_time,100
142,946,wasting_treatment,severe_acute_malnutrition,1.611424,state_person_time,person_time,100
143,946,wasting_treatment,susceptible_to_child_wasting,69.071101,state_person_time,person_time,100


In [47]:
vop.describe(wasting_prevalence)['mean'].unstack().round(2)

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         100        state_person_time baseline                        21.52   
                                                 sqlns                           23.02   
                                                 wasting_treatment               22.41   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                  7.96   
                                                 sqlns                                     6.15   
                                                 wasting_treatment                         6.91   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         100        state_person_time baseline                                1.89   
                                                 sqlns                                   1.46   
                                                 wasting_treatment                       1.60   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                  68.64  
                                                 sqlns                                     69.37  
                                                 wasting_treatment                         69.08

# Check that wasting prevalence is same in all scenarios before interventions start

Yes, looks good.

In [20]:
pt_by_wasting_state_pre_2023 = vop.stratify(
    data.wasting_state_person_time.query("year <= '2022'"), ['wasting_state', 'measure'])
pt_by_wasting_state_pre_2023

,wasting_state,measure,input_draw,scenario,value
0,mild_child_wasting,state_person_time,29,baseline,217034.563997
1,mild_child_wasting,state_person_time,29,sqlns,217034.563997
2,mild_child_wasting,state_person_time,29,wasting_treatment,217034.563997
3,mild_child_wasting,state_person_time,223,baseline,215532.668036
...,...,...,...,...,...
140,susceptible_to_child_wasting,state_person_time,829,wasting_treatment,708511.353867
141,susceptible_to_child_wasting,state_person_time,946,baseline,705681.520876
142,susceptible_to_child_wasting,state_person_time,946,sqlns,705681.520876
143,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,705681.520876


In [21]:
pt_pre_2023 = vop.stratify(data.person_time.query("age != 'all_ages' and year <= '2022'"), ['measure'])
pt_pre_2023

,measure,input_draw,scenario,value
0,person_time,29,baseline,1.024442e+06
1,person_time,29,sqlns,1.024442e+06
2,person_time,29,wasting_treatment,1.024442e+06
3,person_time,223,baseline,1.021514e+06
...,...,...,...,...
32,person_time,829,wasting_treatment,1.024773e+06
33,person_time,946,baseline,1.023324e+06
34,person_time,946,sqlns,1.023324e+06
35,person_time,946,wasting_treatment,1.023324e+06


In [22]:
pop_fraction_by_wasting_state_pre_2023 = vop.ratio(
    pt_by_wasting_state_pre_2023,
    pt_pre_2023,
    strata = [],
    numerator_broadcast='wasting_state'
)
pop_fraction_by_wasting_state_pre_2023

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,0.211856,state_person_time,person_time,1
1,29,baseline,moderate_acute_malnutrition,0.078844,state_person_time,person_time,1
2,29,baseline,severe_acute_malnutrition,0.019271,state_person_time,person_time,1
3,29,baseline,susceptible_to_child_wasting,0.690028,state_person_time,person_time,1
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.209361,state_person_time,person_time,1
141,946,wasting_treatment,moderate_acute_malnutrition,0.079853,state_person_time,person_time,1
142,946,wasting_treatment,severe_acute_malnutrition,0.021189,state_person_time,person_time,1
143,946,wasting_treatment,susceptible_to_child_wasting,0.689598,state_person_time,person_time,1


In [23]:
vop.describe(pop_fraction_by_wasting_state_pre_2023)['mean'].unstack()

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         1          state_person_time baseline                     0.211709   
                                                 sqlns                        0.211709   
                                                 wasting_treatment            0.211709   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                              0.078454   
                                                 sqlns                                 0.078454   
                                                 wasting_treatment                     0.078454   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         1          state_person_time baseline                             0.01877   
                                                 sqlns                                0.01877   
                                                 wasting_treatment                    0.01877   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                               0.691067  
                                                 sqlns                                  0.691067  
                                                 wasting_treatment                      0.691067